# Camera calibration

Notebook is based on OpenCV tutorial (https://docs.opencv.org/4.x/dc/dbb/tutorial_py_calibration.html)


see also for visualization:

https://www.rerun.io/


online viewer:

https://www.rerun.io/viewer


In [ ]:
!wget -O samples.csv https://raw.githubusercontent.com/ant-nik/semares/master/data/stereo-camera-cyl/position.csv

In [ ]:
import cv2
import logging
import io
import numpy
import plotly.express as plte


logger = logging.getLogger(__name__)
# termination criteria
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)


def load_image(payload: any) -> any:
    np_image = numpy.frombuffer(payload, numpy.uint8)
    img = cv2.imdecode(np_image, cv2.IMREAD_COLOR)
    img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)

    return img


def get_chess_corners_from_image(
        image: any, corners_x: int, corners_y: int) -> tuple[any, any]:
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Find the chess board corners
    ret, corners = cv2.findChessboardCorners(gray, (corners_x, corners_y), None)

    # If found, add object points, image points (after refining them)
    if ret != True:
        logger.error("Error, grid (%d, %d) is not found in image",
                     corners_x, corners_y)
        return None, gray

    corners2 = cv2.cornerSubPix(gray, corners, (11, 11), (-1,-1), criteria)
    result_image = cv2.drawChessboardCorners(image, (7,7), corners, True)
    return corners2, result_image


def undistort_image(image: numpy.ndarray, mtx: numpy.ndarray,
                    dist: numpy.ndarray, new_mtx: numpy.ndarray,
                    canvas_w: int, canvas_h: int) -> numpy.ndarray:
    mapx, mapy = cv2.initUndistortRectifyMap(mtx, dist, None, new_mtx,
                                             (canvas_w, canvas_h), 5)
    return cv2.remap(image, mapx, mapy, cv2.INTER_LINEAR)


In [ ]:
import pandas
import numpy
import logging
import functools


logger = logging.getLogger(__name__)


def load_single_image(item: pandas.Series, url_template: str) -> numpy.ndarray:
    item_file = item["image"].lstrip(" ")
    url = url_template.format(item_file)
    response = requests.get(url)
    if response.status_code != 200:
        logger.error("Can't read image %s from url %s", item_file, url)
    return pandas.Series([load_image(response.content), url],
                         index=["image_data", "url"])

In [ ]:
import requests
import logging


logger = logging.getLogger(__name__)


def match_points(
        corners_x: numpy.ndarray,
        corners_y: numpy.ndarray,
        corners: numpy.ndarray,
) -> tuple[numpy.ndarray, numpy.ndarray]:
    objp = numpy.zeros((corners_x*corners_y, 3), numpy.float32)
    objp[:,:2] = numpy.mgrid[0:corners_x, 0:corners_y].T.reshape(-1, 2)
    return objp


def calculate_image_corners(
        row: pandas.Series, ignore_list: str=None, shapes: numpy.ndarray=None,
        image_field="image_data"
) -> pandas.Series:
    result = pandas.Series({
        "corners": None, "chess_image": None, "object_points": None})
    if ignore_list is not None and row.image in ignore_list:
        return result

    corners, image = get_chess_corners_from_image(
        image=numpy.copy(row[image_field]),
        corners_x=row.corners_x,
        corners_y=row.corners_y
    )

    img_shape = (row.image_data.shape[1], row.image_data.shape[0])
    #if shapes != img_shape:
    #    logger.error(
    #        "Image %s has specific shape %s that is not equals to previously shapes: %s",
    #        sample.name, str(img_shape), str(shapes)
    #        )
    #    return pandas.Series([None, None, None], index=["obj_points", "image_points"])

    if corners is None:
        objp = None
        logger.error("Can't find %d, %d conrenrs in %s",
                     row.corners_x, row.corners_y, row.image)
    else:
        objp = match_points(corners_x=row.corners_x,
                            corners_y=row.corners_y,
                            corners=corners)

    result.corners = corners
    result.chess_image = image
    result.object_points= objp

    return result


In [ ]:
import logging


logger = logging.getLogger(__name__)


def undistort_image_from_row(
        row: pandas.Series,
        camera_matrix: numpy.ndarray,
        distortion_matrix: numpy.ndarray,
        new_camera_matrix: numpy.ndarray,
        region: tuple[int, int, int, int]
) -> pandas.Series:
    axis = numpy.float32([[3, 0, 0], [0, 3, 0], [0, 0, -3]]).reshape(-1, 3)
    color = [(255, 0, 0), (0, 255, 0), (0, 0, 255)]
    result = pandas.Series({"undistorted": None, "undistorted_debug": None,
                            "reconstructed_points": None, "imgpoints": None})
    if row["corners"] is None:
        logger.warning("Image %s has no detected chessboard corners, skipping...",
                       row["image"])
        return result

    pre_dst = undistort_image(
        image=row["image_data"],
        mtx=camera_matrix, dist=distortion_matrix,
        new_mtx=new_camera_matrix,
        canvas_w=row["image_data"].shape[1], canvas_h=row["image_data"].shape[0])
    # crop the image
    x, y, w, h = roi
    dst = numpy.copy(pre_dst) # [y:y + h, x:x + w]
    imgpoints, _ = cv2.projectPoints(
        row["object_points"], row["rvec"], row["tvec"],
        camera_matrix, distortion_matrix)
    error = cv2.norm(row["corners"], imgpoints, cv2.NORM_L2)/len(imgpoints)
    #if error < error_threshold:
    #    names.append(sample.name)
    # axis2d, jac = cv2.projectPoints(axis, sample.rvec, sample.tvec, mtx, dist)
    #origin = tuple([int(v) for v in sample.corners[0].ravel()])
    #for i in range(0, 3):
    #    dst = cv2.line(dst, origin, tuple(
    #        [int(v) for v in axis2d[i].ravel()]), color[i], 5)
    lwidth = 2
    dst = cv2.line(dst, (x, y), (x + w, y), (0, 255, 0), lwidth)
    dst = cv2.line(dst, (x + w, y), (x + w, y + h), (0, 255, 0), lwidth)
    dst = cv2.line(dst, (x + w, y + h), (x, y + h), (0, 255, 0), lwidth)
    dst = cv2.line(dst, (x, y + h), (x, y), (0, 255, 0), lwidth)

    result["undistorted_debug"] = dst
    result["undistorted"] = pre_dst
    result["reconstructed_points"] = imgpoints
    result["error"] = error

    return result

def draw_image_sequence(*args, axis=1) -> numpy.ndarray:
    if len(args) == 0:
        return numpy.ndarray()

    w = args[0].shape[1]
    h = args[0].shape[0]
    canvas_shape = list(args[0].shape)
    canvas_shape[axis] = canvas_shape[axis]*len(args)
    canvas = numpy.zeros(canvas_shape)
    for i in range(0, len(args)):
        if axis == 1:
            canvas[:, i*w:(i + 1)*w, :] = args[i]
        else:
            canvas[i*w:(i + 1)*w, :, :] = args[i]

    return canvas

In [ ]:
base_url = "https://raw.githubusercontent.com/ant-nik/semares/master/data/stereo-camera-cyl/{}"
ignore_list = [
    "image75_r.jpg", "image84_r.jpg",
    "image84_l.jpg", "image89_r.jpg",
    "image89_l.jpg", "image91_r.jpg",
    "image91_l.jpg"
]

dataframe = pandas.read_csv("samples.csv", sep=',')
dataframe = dataframe.join(
    dataframe.apply(
        functools.partial(
            load_single_image,
            url_template=base_url),
        axis=1)
)
shapes = dataframe["image_data"].apply(lambda x: x.shape).unique()
if shapes.shape[0] != 1:
    logger.error("More than one unique image sizes were found: %s",
                 str(shapes))
shapes = (shapes[0][1], shapes[0][0])

dataframe = dataframe.join(
    dataframe.apply(
        functools.partial(
            calculate_image_corners,
            ignore_list=ignore_list,
            shapes=None),
        axis=1)
)

filtered = dataframe[~dataframe["image"].isin(ignore_list)]
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(
    filtered["object_points"].tolist(),
    filtered["corners"].tolist(), shapes,
    None, None)
filtered = filtered.assign(rvec=rvecs, tvec=tvecs)
dataframe = dataframe.join(filtered[["rvec", "tvec"]])

w = shapes[0]
h = shapes[1]
newcameramtx, roi = cv2.getOptimalNewCameraMatrix(mtx, dist, (w,h), 1, (w,h))
(roi, newcameramtx)

dataframe = dataframe.join(
    dataframe.apply(
        functools.partial(
            undistort_image_from_row,
            camera_matrix=mtx,
            distortion_matrix=dist,
            new_camera_matrix=newcameramtx,
            region=roi
        ), axis=1
    )
)
x, y, w, h = roi
dataframe.undistorted = dataframe.apply(
    lambda row: row.undistorted[
        y:y + h, x:x + w] if row.undistorted is not None else None
    , axis=1)
dataframe[["image", "error"]]

filtered_undistorted = dataframe[~dataframe["image"].isin(ignore_list)]
dataframe = dataframe.join(
    filtered_undistorted.apply(functools.partial(
        calculate_image_corners, image_field="undistorted"
    ), axis=1), rsuffix="_undistorted")
dataframe.columns

In [ ]:
plte.imshow(dataframe.chess_image.iloc[0]).show()
plte.imshow(dataframe.undistorted.iloc[0]).show()

## Stereo experiments

[stereoCalibrate](https://docs.opencv.org/4.x/d9/d0c/group__calib3d.html#ga9d2539c1ebcda647487a616bdf0fc716) - calculates transformations to locate points of one camera's image on another one based on known pattern.

[stereoRectify](https://docs.opencv.org/4.x/d9/d0c/group__calib3d.html#ga617b1685d4059c6040827800e72ad2b6) - calculates a transformation to represent images from stereo pair on a single plane.

In [ ]:
stereodata = dataframe[dataframe.type == "r"].merge(
    dataframe[dataframe.type == "l"], on="no", suffixes=["_right", "_left"])
stereodata.columns

In [ ]:
stereo_calib_data = stereodata[
    (~stereodata.corners_left.isnull()) & (~stereodata.corners_right.isnull())]
stereo_calib_data = stereo_calib_data[
    (stereo_calib_data.object_points_left.apply(lambda x: x.shape)
     ==stereo_calib_data.object_points_right.apply(lambda x: x.shape))
]
len(stereo_calib_data)

In [ ]:
"""
calib_data = cv2.stereoCalibrate(
    stereo_calib_data.object_points_undistorted_left.iloc[0],
    stereo_calib_data.corners_undistorted_left.iloc[0],
    stereo_calib_data.corners_undistorted_right.iloc[0],

)
"""

In [ ]:
def calculate_descriptors(left_image: any, right_image: any
                          ) -> tuple[any, any]:
    sift = cv2.SIFT_create()
    # find the keypoints and descriptors with SIFT
    kp1, des1 = sift.detectAndCompute(left_image, None)
    kp2, des2 = sift.detectAndCompute(right_image, None)

    # FLANN parameters
    FLANN_INDEX_KDTREE = 1
    index_params = dict(algorithm=FLANN_INDEX_KDTREE, trees=5)
    search_params = dict(checks=50)

    flann = cv2.FlannBasedMatcher(index_params, search_params)
    matches = flann.knnMatch(des1, des2, k=2)

    pts1 = []
    pts2 = []

    # ratio test as per Lowe's paper
    for i,(m,n) in enumerate(matches):
        if m.distance < 0.8*n.distance:
            pts2.append(kp2[m.trainIdx].pt)
            pts1.append(kp1[m.queryIdx].pt)

    return pts1, pts2

In [ ]:
x, y, w, h = roi
num = 0
left_pts, right_pts = calculate_descriptors(left_image=stereodata.undistorted_left.iloc[num],
                                            right_image=stereodata.undistorted_left.iloc[num])

In [ ]:
def draw_matched_points(
    left_image: numpy.ndarray,
    right_image: numpy.ndarray,
    left_points: numpy.ndarray,
    right_points: numpy.ndarray
) -> None:
    canvas_shape = list(left_image.shape)
    canvas_shape[1] = canvas_shape[1]*2
    canvas = numpy.zeros(canvas_shape)
    canvas[:, 0:left_image.shape[1], :] = left_image
    canvas[:, left_image.shape[1]:, :] = right_image
    for p_left, p_right in zip(left_points, right_points):
        color = tuple(numpy.random.randint(0, 255, 3).tolist())
        left_point = tuple(map(int, p_left))
        canvas = cv2.circle(canvas, left_point, 5, color,-1)
        p_right_shifted = list(p_right)
        p_right_shifted[0] = p_right_shifted[0] + left_image.shape[1]
        right_point = tuple(map(int, p_right_shifted))
        canvas = cv2.circle(canvas, right_point, 5, color,-1)
        canvas = cv2.line(canvas, left_point, right_point, (0, 255, 0), 1)
    fig = plte.imshow(canvas)
    fig.show()

In [ ]:
draw_matched_points(
    left_image=stereodata.undistorted_left.iloc[num],
    right_image=stereodata.undistorted_right.iloc[num],
    left_points=left_pts,
    right_points=right_pts
)

In [ ]:
def calculate_fundamental_matrix(
    p1: numpy.ndarray, p2: numpy.ndarray
) -> tuple[numpy.ndarray, numpy.ndarray]:
    p1 = numpy.int32(p1)
    p2 = numpy.int32(p2)
    return cv2.findFundamentalMat(p1, p2, cv2.FM_LMEDS)

In [ ]:
Fm, mask = calculate_fundamental_matrix(
    p1=left_pts, p2=right_pts
)
left_pts_filtered = numpy.array(left_pts)[mask.ravel()==1]
right_pts_filtered = numpy.array(right_pts)[mask.ravel()==1]

In [ ]:
draw_matched_points(
    left_image=stereodata.undistorted_left.iloc[num],
    right_image=stereodata.undistorted_right.iloc[num],
    left_points=left_pts_filtered,
    right_points=right_pts_filtered
)

In [ ]:
stereo = cv2.StereoBM.create(numDisparities=16, blockSize=15)
first_left_gray = cv2.cvtColor(stereodata.undistorted_left.iloc[num], cv2.COLOR_BGR2GRAY)
first_right_gray = cv2.cvtColor(stereodata.undistorted_right.iloc[num], cv2.COLOR_BGR2GRAY)
disparity = stereo.compute(first_left_gray, first_right_gray)
plte.imshow(disparity).show()

## Web camera experiments

In [ ]:
!wget -O real-samples.csv https://raw.githubusercontent.com/ant-nik/semares/master/data/stereo-camera-real-test/position.csv

In [ ]:
base_url = "https://raw.githubusercontent.com/ant-nik/semares/master/data/stereo-camera-real-test/{}"
dataframe = pandas.read_csv("real-samples.csv", sep=',')
dataframe = dataframe.join(
    dataframe.apply(
        functools.partial(
            load_single_image,
            url_template=base_url),
        axis=1)
)
plte.imshow(draw_image_sequence(dataframe.image_data.iloc[1], dataframe.image_data.iloc[0]))

In [ ]:
left_pts, right_pts = calculate_descriptors(left_image=dataframe.image_data.iloc[1],
                                            right_image=dataframe.image_data.iloc[0])

In [ ]:
draw_matched_points(
    left_image=dataframe.image_data.iloc[1],
    right_image=dataframe.image_data.iloc[0],
    left_points=left_pts,
    right_points=right_pts
)

In [ ]:
Fm, mask = calculate_fundamental_matrix(
    p1=left_pts, p2=right_pts
)
left_pts_filtered = numpy.array(left_pts)[mask.ravel()==1]
right_pts_filtered = numpy.array(right_pts)[mask.ravel()==1]

In [ ]:
draw_matched_points(
    left_image=dataframe.image_data.iloc[1],
    right_image=dataframe.image_data.iloc[0],
    left_points=left_pts_filtered,
    right_points=right_pts_filtered
)